<a href="https://colab.research.google.com/github/Yi-Wei-Lin/Tibame_AI_Project/blob/main/%E7%94%98%E8%97%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
# train_df = pd.read_csv("banana.csv", encoding="utf-8")
train_df = pd.read_excel("gan.xlsx")

# Date 民國 to 西元
def date_df(d):
  d = d.split("/")
  year = str(int(d[0])+1911)
  month = (d[1])
  day = (d[2])
  d = (f"{year}-{month}-{day}")
  return d
date_df = pd.DataFrame({"Date":train_df["Date"].apply(date_df)})
train_df.update(date_df)

# Month method
def Month(m):
  Month_data = int(m.split("-")[1])
  return Month_data
train_df["Month"] = train_df["Date"].apply(Month)

# Week_day method
def week_day(d):
  d = d.split("-")
  year = int(d[0])
  month = int(d[1])
  day = int(d[2])
  temp = pd.Timestamp(f"{year}-{month}-{day}")
  week_day = temp.dayofweek + 1 #temp.dayofweek 0(禮拜一) 開始, +1是因為比較直接禮拜幾
  return week_day
train_df["Week_day"] = train_df["Date"].apply(week_day)
train_df["Year"] = train_df["Date"].apply(lambda d:int(d.split("-")[0]))

# Rest_market
market_df = pd.DataFrame({"Market":train_df["Market"].apply(lambda t:t.split(" ")[1])})
train_df.update(market_df)

# Volume chnage from str to int
# train_df["Volume"] = train_df["Volume"].apply(lambda x:int(x.replace(",","")))

# Up_price change from onj to float
# train_df["Up_price"] = train_df["Up_price"].apply(lambda x:float(x))
market_df["Market"]
train_df

,Date,Market,Product,Up_price,Mid_price,Low_price,Avg_price,Volume,Month,Week_day,Year
0,2000-01-02,台北二,LA1 甘藍 初秋,10.4,7.2,5.5,7.5,48587.0,1,7,2000
1,2000-01-03,台北二,LA1 甘藍 初秋,8.7,5.7,4.4,6.1,39013.0,1,1,2000
2,2000-01-04,台北二,LA1 甘藍 初秋,7.2,4.8,3.0,4.9,31258.0,1,2,2000
3,2000-01-05,台北二,LA1 甘藍 初秋,9.4,6.7,4.3,6.8,24181.0,1,3,2000
4,2000-01-06,台北二,LA1 甘藍 初秋,9.2,7.4,4.5,7.2,25484.0,1,4,2000
...,...,...,...,...,...,...,...,...,...,...,...
89946,2021-06-01,花蓮市,LA1 甘藍 初秋,32.0,32.0,30.0,31.6,1041.0,6,2,2021
89947,2021-06-02,花蓮市,LA1 甘藍 初秋,36.0,35.3,32.9,35.0,810.0,6,3,2021
89948,2021-06-03,花蓮市,LA1 甘藍 初秋,34.4,34.0,33.8,34.1,768.0,6,4,2021
89949,2021-06-04,花蓮市,LA1 甘藍 初秋,16.0,16.0,16.0,16.0,54.0,6,5,2021


In [2]:
train_df["Date"] = pd.to_datetime(train_df["Date"])
train_df["Date"]

0       2000-01-02
1       2000-01-03
2       2000-01-04
3       2000-01-05
4       2000-01-06
           ...    
89946   2021-06-01
89947   2021-06-02
89948   2021-06-03
89949   2021-06-04
89950   2021-06-08
Name: Date, Length: 89951, dtype: datetime64[ns]

In [3]:
train_df
train_df.dtypes

Date         datetime64[ns]
Market               object
Product              object
Up_price            float64
Mid_price           float64
Low_price           float64
Avg_price           float64
Volume              float64
Month                 int64
Week_day              int64
Year                  int64
dtype: object

In [4]:
# 補市場休市欄位
market_list = train_df["Market"].value_counts().index.tolist()
m_total = []
for i,mark_name in enumerate(market_list):
  a = train_df["Market"] == mark_name
  start_time = train_df[a].reset_index(drop=True)["Date"][0]
  t_index = pd.date_range(start_time, '2021-06-19', freq='D') #start_time by 市場別
  T_df = pd.DataFrame(t_index, columns=['Date'])
  m = train_df[a].merge(T_df, how="right",sort=True)
  most = m.value_counts().idxmax()
  m["Market"]=m["Market"].fillna(most[1])
  m["Product"]=m["Product"].fillna(most[2])
  na = m["Volume"].isna()
  m["Month"] = m["Date"].apply(lambda x:int(str(x).split("-")[1]))
  m["Week_day"] = m["Date"].apply(lambda x:pd.Timestamp(str(x).split()[0]).dayofweek+1)
  m["Year"] = m["Date"].apply(lambda x:int(str(x).split("-")[0]))
  import math
  def restday(s):
    if s > 0:
      return 0
    else:
      return 1
  m["Rest_day"] = m["Volume"].apply(restday) #判斷 restday 0 or 1
  globals()[f"m{i}"] = m
  m_total.append(globals()[f"m{i}"])
print(m.dtypes)
len(market_list)
final_data = pd.concat(m_total).reset_index(drop=True)

Date         datetime64[ns]
Market               object
Product              object
Up_price            float64
Mid_price           float64
Low_price           float64
Avg_price           float64
Volume              float64
Month                 int64
Week_day              int64
Year                  int64
Rest_day              int64
dtype: object


In [6]:
final_data.to_csv("Cabbage.csv", encoding="utf-8", index=False)
final_data

,Date,Market,Product,Up_price,Mid_price,Low_price,Avg_price,Volume,Month,Week_day,Year,Rest_day
0,2000-01-01,西螺鎮,LA1 甘藍 初秋,9.4,6.9,4.5,6.9,255395.0,1,6,2000,0
1,2000-01-02,西螺鎮,LA1 甘藍 初秋,9.8,7.1,4.9,7.2,247275.0,1,7,2000,0
2,2000-01-03,西螺鎮,LA1 甘藍 初秋,9.9,7.2,4.9,7.3,242391.0,1,1,2000,0
3,2000-01-04,西螺鎮,LA1 甘藍 初秋,9.5,7.0,4.7,7.0,248793.0,1,2,2000,0
4,2000-01-05,西螺鎮,LA1 甘藍 初秋,8.4,6.5,4.4,6.5,241955.0,1,3,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
111166,2021-06-15,板橋區,LA1 甘藍 初秋,NaN,NaN,NaN,NaN,NaN,6,2,2021,1
111167,2021-06-16,板橋區,LA1 甘藍 初秋,NaN,NaN,NaN,NaN,NaN,6,3,2021,1
111168,2021-06-17,板橋區,LA1 甘藍 初秋,21.0,13.8,8.0,14.1,36960.5,6,4,2021,0
111169,2021-06-18,板橋區,LA1 甘藍 初秋,20.1,12.2,7.0,12.8,27821.8,6,5,2021,0
